In [10]:
import pandas as pd
import json
from neo4j import GraphDatabase

# Neo4j connection config
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

In [ ]:
with open("../data/companyDetailsByKodeEmiten.json", "r", encoding="utf-8") as f:
    stocks_profile = json.load(f)  # Make sure this returns a list of dicts

len(pd.read_json('..//data/companyDetailsByKodeEmiten.json').columns)

print(type(stocks_profile))         # should be <class 'list'>
# print(type(stocks_profile[0]))      # should be <class 'dict'>

In [ ]:
import re

def clean_indonesian_name(name):
    original_name = name
    # Define known titles, degrees, and honorifics (add more as needed)
    noise_tokens = {
        'dr', 'drs', 'h', 'ir', 'prof', 'kh', 'hj', 'hrh', 'mr', 'mrs', 'ms',  # prefixes
        'sh', 'mh', 'phd', 'spd', 'mpd', 'se', 'mm', 'msi', 'skom', 'st', 'mt', 'mkom', 'pm', 'bsc'  # suffixes
    }

    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)  # remove punctuation
    tokens = name.split()
    
    # Remove known titles and single-letter fragments (initials)
    tokens = [t for t in tokens if t not in noise_tokens and len(t) > 1]
    
    return original_name
    # return ' '.join(tokens)

# Example usage
names = [
    "Dr. M. Arsjad Rasjid PM BSC",
    "m arsjad rasjid pm",
    "MR. Joko Widodo, S.H., M.H.",
    "Ir. Sri Mulyani Indrawati, M.Si.",
    "Mrs. Megawati S. Soekarnoputri",
    "Hj. Tri Rismaharini"
]

cleaned = [clean_indonesian_name(n) for n in names]
for original, c in zip(names, cleaned):
    print(f"{original}  -->  {c}")


In [ ]:
# Cypher command to delete all data
cypher_query = "MATCH (n) DETACH DELETE n"

def delete_all_data():
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        session.run(cypher_query)
    driver.close()
    print("All data deleted.")

delete_all_data()

In [ ]:
# Load JSON data
with open("../data/companyDetailsByKodeEmiten.json", "r", encoding="utf-8") as f:
    stocks_profile = dict(list(json.load(f).items())[:])

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def ingest_stock(tx, stock):
    profile = stock['Profiles'][0]
    kode = profile["KodeEmiten"]

    # Create/Update Company node with enriched data
    tx.run("""
        MERGE (c:Company {kode: $kode})
        SET c.name = $kode,
            c.companyName = $name,
            c.industry = $industry,
            c.subIndustry = $sub_industry,
            c.sector = $sector,
            c.subSector = $sub_sector,
            c.website = $website,
            c.email = $email,
            c.phone = $telepon,
            c.fax = $fax,
            c.address = $alamat,
            c.npwp = $npwp,
            c.listingBoard = $papan,
            c.listingDate = date($tanggal_pencatatan),
            c.businessActivity = $kegiatan_usaha
    """, kode=kode,
         name=profile["NamaEmiten"],
         industry=profile["Industri"],
         sub_industry=profile.get("SubIndustri"),
         sector=profile["Sektor"],
         sub_sector=profile.get("SubSektor"),
         website=profile["Website"],
         email=profile.get("Email"),
         telepon=profile.get("Telepon"),
         fax=profile.get("Fax"),
         alamat=profile.get("Alamat"),
         npwp=profile.get("NPWP"),
         papan=profile.get("PapanPencatatan"),
         tanggal_pencatatan=profile.get("TanggalPencatatan", "")[:10],
         kegiatan_usaha=profile.get("KegiatanUsahaUtama")
    )

    # Directors
    for d in stock.get("Direktur", []):
        tx.run("""
            MERGE (d:Insider {name: $name})
            WITH d
            MATCH (c:Company {kode: $kode})
            MERGE (d)-[:DIRECTOR_OF {jabatan: $jabatan, afiliasi: $afiliasi}]->(c)
        """, name=clean_indonesian_name(d["Nama"]), jabatan=d["Jabatan"], afiliasi=d.get("Afiliasi", False), kode=stock['Profiles'][0]["KodeEmiten"])

    # Commissioners
    for k in stock.get("Komisaris", []):
        tx.run("""
            MERGE (k:Insider {name: $name})
            WITH k
            MATCH (c:Company {kode: $kode})
            MERGE (k)-[:COMMISSIONER_OF {jabatan: $jabatan, independen: $independen}]->(c)
        """, name=clean_indonesian_name(k["Nama"]), jabatan=k["Jabatan"], independen=k.get("Independen", False), kode=stock['Profiles'][0]["KodeEmiten"])

    # Corporate Secretary
    for s in stock.get("Sekretaris", []):
        tx.run("""
            MERGE (sec:Insider {name: $name})
            WITH sec
            MATCH (c:Company {kode: $kode})
            MERGE (sec)-[:CORPORATE_SECRETARY_OF {
                phone: $phone, email: $email, fax: $fax
            }]->(c)
        """, name=clean_indonesian_name(s["Nama"]),
             phone=s.get("Telepon"),
             email=s.get("Email"),
             fax=s.get("Fax"),
             kode=kode
        )

    # Audit Committee
    for a in stock.get("KomiteAudit", []):
        tx.run("""
            MERGE (ac:Insider {name: $name})
            WITH ac
            MATCH (c:Company {kode: $kode})
            MERGE (ac)-[:AUDIT_COMMITTEE_MEMBER_OF {jabatan: $jabatan}]->(c)
        """, name=clean_indonesian_name(a["Nama"]),
             jabatan=a["Jabatan"],
             kode=kode
        )

    # Shareholders
    for s in stock.get("PemegangSaham", []):
        tx.run("""
            MERGE (s:Insider {name: $name})
            WITH s
            MATCH (c:Company {kode: $kode})
            MERGE (s)-[:OWNS {jumlah: $jumlah, kategori: $kategori, pengendali: $pengendali, persentase: $persentase}]->(c)
        """,
            jumlah=s.get("Jumlah"),  
            kategori=s.get("Kategori"),
            name=clean_indonesian_name(s["Nama"]), 
            pengendali=s.get("Pengendali"),
            persentase=s.get("Persentase"),  
            kode=stock['Profiles'][0]["KodeEmiten"]
        )
    # Subsidiaries (AnakPerusahaan)
    for a in stock.get("AnakPerusahaan", []):
        tx.run("""
            MERGE (s:Subsidiary {name: $name})
            SET s.bidangUsaha = $bidang_usaha, s.lokasi = $lokasi, s.jumlahAset = $jumlah_aset,
                s.satuan = $satuan, s.statusOperasi = $status_operasi, s.tahunKomersil = $tahun_komersil,
                s.mataUang = $mata_uang
            WITH s
            MATCH (c:Company {kode: $kode})
            MERGE (s)-[:SUBSIDIARY_OF {persentase: $persentase}]->(c)
        """, name=a["Nama"], bidang_usaha=a["BidangUsaha"], lokasi=a["Lokasi"], jumlah_aset=a["JumlahAset"],
            satuan=a["Satuan"], status_operasi=a["StatusOperasi"], tahun_komersil=a["TahunKomersil"],
            mata_uang=a["MataUang"], persentase=a["Persentase"], kode=stock['Profiles'][0]["KodeEmiten"])

    # # Dividends (Dividen)
    # for d in stock.get("Dividen", []):
    #     tx.run("""
    #         MATCH (c:Company {kode: $kode})
    #         MERGE (div:Dividend {tahunBuku: $tahun_buku, jenis: $jenis})
    #         SET div.nama = $nama, div.totalSahamBonus = $total_saham_bonus, 
    #             div.cashDividenPerSaham = $cash_per_saham, div.cashDividenPerSahamMU = $cash_per_saham_mu,
    #             div.cashDividenTotal = $cash_total, div.cashDividenTotalMU = $cash_total_mu,
    #             div.tanggalCum = date($tanggal_cum), div.tanggalEx = date($tanggal_ex),
    #             div.tanggalDPS = datetime($tanggal_dps), div.tanggalPembayaran = date($tanggal_pembayaran),
    #             div.rasio1 = $rasio1, div.rasio2 = $rasio2
    #         MERGE (c)-[:PAID_DIVIDEND]->(div)
    #     """, kode=d["Nama"], nama=d["Nama"], jenis=d["Jenis"], tahun_buku=d["TahunBuku"],
    #         total_saham_bonus=d["TotalSahamBonus"], cash_per_saham=d["CashDividenPerSaham"],
    #         cash_per_saham_mu=d["CashDividenPerSahamMU"], cash_total=d["CashDividenTotal"],
    #         cash_total_mu=d["CashDividenTotalMU"], tanggal_cum=d["TanggalCum"][:10],
    #         tanggal_ex=d["TanggalExRegulerDanNegosiasi"][:10], tanggal_dps=d["TanggalDPS"],
    #         tanggal_pembayaran=d["TanggalPembayaran"][:10], rasio1=d["Rasio1"], rasio2=d["Rasio2"])

    # # Bonds and Sukuk
    # for b in stock.get("BondsAndSukuk", []):
    #     tx.run("""
    #         MERGE (bond:Bond {isin: $isin})
    #         SET bond.namaEmisi = $nama_emisi, bond.listingDate = date($listing_date),
    #             bond.matureDate = date($mature_date), bond.rating = $rating, bond.nominal = $nominal,
    #             bond.margin = $margin, bond.waliAmanat = $wali_amanat
    #         WITH bond
    #         MATCH (c:Company {kode: $kode})
    #         MERGE (c)-[:ISSUED]->(bond)
    #     """, isin=b["ISINCode"], nama_emisi=b["NamaEmisi"], listing_date=b["ListingDate"][:10],
    #         mature_date=b["MatureDate"][:10], rating=b["Rating"], nominal=b["Nominal"],
    #         margin=b["Margin"], wali_amanat=b["WaliAmanat"], kode=b["KodeEmiten"])

    # # Issued Bond Details
    # for i in stock.get("IssuedBond", []):
    #     tx.run("""
    #         MERGE (i:IssuedBond {id: $id})
    #         SET i.namaSeri = $nama_seri, i.amortisasiValue = $amortisasi_value,
    #             i.sinkingFund = $sinking_fund, i.couponDetail = $coupon_detail,
    #             i.couponPaymentDetail = date($coupon_payment_detail),
    #             i.matureDate = date($mature_date)
    #         WITH i
    #         MATCH (c:Company {kode: $kode})
    #         MERGE (c)-[:ISSUED_DETAIL]->(i)
    #     """, id=i["id"], nama_seri=i["NamaSeri"], amortisasi_value=i["AmortisasiValue"],
    #         sinking_fund=i["SinkingFund"], coupon_detail=i["CouponDetail"],
    #         coupon_payment_detail=i["CouponPaymentDetail"][:10],
    #         mature_date=i["MatureDate"][:10], kode=i["KodeEmiten"])

# Ingest all stocks_profile
with driver.session() as session:
    for ticker, stock_data in stocks_profile.items():
        # print(ticker, stock_data['Profiles'][0][0]["KodeEmiten"])
        session.execute_write(ingest_stock, stock_data) 

print("Ingestion complete.")
driver.close()


In [ ]:
with open("../data/companySummaryByKodeEmiten.json", "r", encoding="utf-8") as f:
    stocks_summary = json.load(f)['data']  # Make sure this returns a list of dicts

cypher_query = """
UNWIND $batch AS row
MERGE (c:Company {kode: row.StockCode})
MERGE (s:TradeDay {
    date: date(split(row.Date, "T")[0]),
    kode: row.StockCode
})
SET 
    s.name = toString(date(split(row.Date, "T")[0])) + "|" + row.StockCode,
    s.idstocksummary=row.IDStockSummary,
    s.stockname=row.StockName,
    s.remarks=row.Remarks,
    s.previous=row.Previous,
    s.openprice=row.OpenPrice,
    s.firsttrade=row.FirstTrade,
    s.high=row.High,
    s.low=row.Low,
    s.close=row.Close,
    s.change=row.Change,
    s.volume=row.Volume,
    s.value=row.Value,
    s.frequency=row.Frequency,
    s.indexindividual=row.IndexIndividual,
    s.offer=row.Offer,
    s.offervolume=row.OfferVolume,
    s.bid=row.Bid,
    s.bidvolume=row.BidVolume,
    s.listedshares=row.ListedShares,
    s.tradebleshares=row.TradebleShares,
    s.weightforindex=row.WeightForIndex,
    s.foreignsell=row.ForeignSell,
    s.foreignbuy=row.ForeignBuy,
    s.delistingdate=row.DelistingDate,
    s.nonregularvolume=row.NonRegularVolume,
    s.nonregularvalue=row.NonRegularValue,
    s.nonregularfrequency=row.NonRegularFrequency
MERGE (c)-[:HAS_TRADE_DAY]->(s)
"""

# Neo4j session
def insert_data(uri, user, password, data):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        session.execute_write(lambda tx: tx.run(cypher_query, batch=data))
    driver.close()

# Run ingestion
insert_data(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, stocks_summary)
print("Stock summary data ingested successfully.")

In [ ]:
MATCH (c:Company)-[]-(i:Insider)
WHERE toLower(i.name) CONTAINS 'lo kheng hong'
RETURN c, i

MATCH (i:Insider)
WHERE toLower(i.name) CONTAINS 'jerry ng'
MATCH (i)-[r1:OWNS|DIRECTOR_OF|COMMISSIONER_OF]->(c:Company)
OPTIONAL MATCH (s:Subsidiary)-[r2:SUBSIDIARY_OF]->(c)
RETURN DISTINCT i, r1, c, r2, s

// Delete all StockSummary nodes and their relationships
MATCH (s:TradeDay)<-[r:HAS_TRADE_DAY]-()
DELETE r, s;

MATCH (n)
UNWIND labels(n) AS label
UNWIND keys(n) AS key
RETURN label, collect(DISTINCT key) AS properties
ORDER BY label;


MATCH ()-[r]->()
WITH type(r) AS relType, keys(r) AS keysList
UNWIND keysList AS key
RETURN relType, collect(DISTINCT key) AS properties
ORDER BY relType;

MATCH (i:Insider)-[owns:OWNS]->(c:Company)-[:HAS_TRADE_DAY]->(td:TradeDay)
WITH i, owns.jumlah AS sharesOwned, c, td
WITH i, c, sharesOwned, td.close AS latestClosePrice
WITH i, sum(sharesOwned * latestClosePrice) AS totalValue
where totalValue > 0 and  not (i.name CONTAINS 'masyarakat') and not (i.name CONTAINS 'pt ') and not (i.name CONTAINS ' ltd') 
and not (i.name CONTAINS ' limited') and not (i.name CONTAINS ' pt') and not (i.name CONTAINS ' bv') and not (i.name CONTAINS 'saham') and not (i.name CONTAINS 'pengendali')
RETURN i.name AS InsiderName, totalValue
ORDER BY totalValue DESC

